In [1]:
from to_objects import get_objects
from rdflib import Graph

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'
DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage
DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:hpflex#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:hpflex#', with graph: 'd9c357e9-9d52-4fc6-9fc7-5e8768a49ed6'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'd9c357e9-9d52-4fc6-9fc7-5e8768a49ed6' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hvac-zone'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '087872e0-a5f4-4bd5-8b7c-7c7d54b331b2' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating data

In [2]:
g = Graph()
g.parse('tutorial/bldg2.ttl')

<Graph identifier=N53e48aec659347258d389293c84d9db4 (<class 'rdflib.graph.Graph'>)>

In [3]:
zones = get_objects(g)

In [4]:
zone1 = zones[0]
zone1.spaces[0].area

Value(value=1500.0, unit='FT2')

In [5]:
zones[0].spaces

[Space(name='urn:hpflex/hpflex_demo#space_1_1', area=Value(value=1500.0, unit='FT2'))]

# Drafting

In [6]:
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from rdflib import Graph, Literal, Namespace, URIRef
from semantic_mpc_interface.namespaces import * 
from semantic_mpc_interface.utils import * 
from buildingmotif.ingresses import CSVIngress, TemplateIngress, Record

In [7]:
bm = BuildingMOTIF("sqlite://")
model = Model.create(HPF)


DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:hpflex#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:hpflex#', with graph: 'ba691fdc-6b9f-4be7-ab1d-8cdeceacff2a'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'ba691fdc-6b9f-4be7-ab1d-8cdeceacff2a' in database with: 1 triples


In [8]:
tmp = Library.load(directory='src/semantic_mpc_interface/templates/brick-templates')

DEBUG:buildingmotif.database.table_connection:Creating database template: 'hvac-zone'


DEBUG:buildingmotif.database.graph_connection:Creating graph: '3a2e1bf3-e5f5-420f-8ee1-9c9d10ca7a0c' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '881f7b96-c895-4278-94da-9f63640f66a5' in database with: 2 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'window'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'f7ca6f0c-a26f-4654-aca3-ac6b105db039' in database with: 4 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hp-rtu'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'daf1f9ac-317d-4148-9050-6d90cd6073cb' in database with: 6 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'tstat'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '597d57d3-b064-48ee-9732-f94bc3f1621a' in database with: 6 triples
DEBUG:buildingmotif.datab

In [9]:
zone = tmp.get_template_by_name('hvac-zone')

In [10]:
zone.optional_args

['space_name', 'window_name', 'tstat_name']

In [11]:
for dependency in zone.get_dependencies():
    zone.remove_dependency(dependency.template)
    print(zone.inline_dependencies().body.serialize(format = 'ttl'))

DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '1' and: '2'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '1' and: '3'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '1' and: '5'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '1' and: '14'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '1' and: '15'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '1' and: '16'


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <http://qudt.org/schema/qudt/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:___param___#tstat_name> a brick:Thermostat ;
    brick:hasLocation <urn:___param___#name> ;
    brick:hasPoint <urn:___param___#tstat_name-active>,
        <urn:___param___#tstat_name-setpoint_deadband>,
        <urn:___param___#tstat_name-tolerance> ;
    brick:operationalStageCount <urn:___param___#tstat_name-stage_count> ;
    brick:resolution <urn:___param___#tstat_name-resolution> .

<urn:___param___#name> a brick:HVAC_Zone ;
    brick:hasPart <urn:___param___#space_name>,
        <urn:___param___#window_name> .

<urn:___param___#tstat_name-active> a brick:Availability_Status ;
    brick:value <urn:___param___#tstat_name-active-value> .

<urn:___param___#tstat_name-resolution> ns1:hasUnit <urn:___param___#tstat_name-resolution-unit> ;
    ns1:isDeltaQuantity true ;
    brick:value <urn:___param___#tstat_name-resolution-valu

In [12]:
tstat = tmp.get_template_by_name('tstat')
for dependency in tstat.get_dependencies():
    tstat.remove_dependency(dependency.template)
    print(tstat.inline_dependencies().body.serialize(format = 'ttl'))

DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '5' and: '36'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '5' and: '37'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '5' and: '38'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '5' and: '39'
DEBUG:buildingmotif.database.table_connection:Deleting depencency from templates with ids: '5' and: '40'


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <http://qudt.org/schema/qudt/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:___param___#name> a brick:Thermostat ;
    brick:hasPoint <urn:___param___#active>,
        <urn:___param___#setpoint_deadband>,
        <urn:___param___#tolerance> ;
    brick:operationalStageCount <urn:___param___#stage_count> ;
    brick:resolution <urn:___param___#resolution> .

<urn:___param___#active> a brick:Availability_Status ;
    brick:value <urn:___param___#active-value> .

<urn:___param___#resolution> ns1:hasUnit <urn:___param___#resolution-unit> ;
    ns1:isDeltaQuantity true ;
    brick:value <urn:___param___#resolution-value> .

<urn:___param___#setpoint_deadband> a brick:Temperature_Deadband_Setpoint ;
    ns1:hasQuantityKind <http://qudt.org/vocab/quantitykind/Temperature> ;
    ns1:hasUnit <urn:___param___#setpoint_deadband-unit> ;
    ns1:isDeltaQuantity true ;
    brick:value <urn:___param___#setpoint_deadba

In [13]:
zone_in = zone.inline_dependencies()

In [14]:
display(zone_in.all_parameters)

{'name'}

In [15]:
print(zone_in.body.serialize(format = 'ttl'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a brick:HVAC_Zone .




In [16]:
def get_var_name(graph, node):
    pre, ns, local = graph.compute_qname(node)
    if PARAM == ns:
        q_n = f"?{local}".replace('-','_')
    else:
        q_n = convert_to_prefixed(node, graph).replace('-','_')
    return q_n

In [17]:
def make_where(graph):
    where = []
    for s,p,o in graph.triples((None,None,None)):
        # print(s,p,o)
        qs = get_var_name(graph, s)
        qo = get_var_name(graph, o)
        qp = convert_to_prefixed(p, graph).replace('-','_')
        where.append(f"{qs} {qp} {qo} .")
    return "\n".join(where)

In [18]:
def get_query(graph):
     where = make_where(graph)
     prefixes = get_prefixes(graph)
     query = f"""{prefixes}\nSELECT DISTINCT * WHERE {{ {where} }}"""
     return query

In [19]:
query = get_query(zone_in.body)

In [20]:
print(query)

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

In [21]:
g = Graph()
g.parse('tutorial/bldg2.ttl')

<Graph identifier=Nceab15ba2291437395b7b978fc17e1bc (<class 'rdflib.graph.Graph'>)>

In [22]:
df = query_to_df(query, g)

In [23]:
class Value:
    unit: str
    value: float

In [24]:
df

,name
0,urn:hpflex/hpflex_demo#zone_1
1,urn:hpflex/hpflex_demo#zone_2
2,urn:hpflex/hpflex_demo#zone_3
3,urn:hpflex/hpflex_demo#zone_4


In [25]:
zone_in.generate_csv('test.csv')

In [26]:
zone

Template(_id=1, _name='hvac-zone', body=<Graph identifier=3a2e1bf3-e5f5-420f-8ee1-9c9d10ca7a0c (<class 'rdflib.graph.Graph'>)>, optional_args=['space_name', 'window_name', 'tstat_name'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x104fffc70>)

In [27]:
zone_part = zone_in.evaluate({p:HPF[p] for p in zone_in.all_parameters if p.split('_')[-1] == 'name'})

In [28]:
zone_part.generate_csv('test.csv')
# dependencies are showing up in headers for some reason - can just remove these. 
# dependencies = zone_part.get_dependencies()

# df = pd.read_csv('test.csv')

# remove_cols = [dependency.template.name for dependency in zone.get_dependencies()]
# df = df.drop(columns=remove_cols)

# # Save the new CSV
# df.to_csv('test.csv', index=False)


AttributeError: 'Graph' object has no attribute 'generate_csv'

In [ ]:
site = tmp.get_template_by_name('site')
site.inline_dependencies().generate_csv('test.csv')

In [1]:
from semantic_mpc_interface import HPFlexSurvey

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


In [2]:
s = HPFlexSurvey('test_site','test_build','.', overwrite=True)

DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage
DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'brick-templates'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'brick-templates'


DEBUG:buildingmotif.database.table_connection:Creating database template: 'hvac-zone'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '05dcf986-4340-4b7a-a787-700eca3f5e3d' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '816b97e0-b1d7-47eb-b2c5-a4eea9d45fdc' in database with: 2 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'window'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'e7adb1a6-a818-4c26-80da-59c3701ace33' in database with: 4 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'hp-rtu'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '2aabc5a9-d3a8-4a19-b2b1-ca9ea763963a' in database with: 6 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'tstat'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '819a2

Failed to load templates from /Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/src/semantic_mpc_interface/templates/brick-templates: The name parameter is required for the dependency 'hp-rtu'.


ValueError: The name parameter is required for the dependency 'hp-rtu'.

In [3]:
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])

{'site_id': 'test_site', 'hvac_type': 'hp-rtu', 'empty_params': {'hvac-zone': [], 'space': ['area_name'], 'hp-rtu': ['cooling_COP_name', 'heating_capacity_name', 'cooling_capacity_name', 'heating_COP_name'], 'tstat': [], 'window': ['azimuth_name', 'area_name', 'tilt_name']}, 'param_mapping': {'hvac-zone': {}, 'space': {'area': 'area_value'}, 'hp-rtu': {'heating_capacity': 'heating_capacity_value', 'cooling_capacity': 'cooling_capacity_value', 'heating_COP': 'heating_COP_value', 'cooling_COP': 'cooling_COP_value'}, 'tstat': {}, 'window': {'tilt': 'tilt_value', 'azimuth': 'azimuth_value', 'area': 'area_value'}}, 'hvacs_feed_hvacs': {}, 'hvacs_feed_zones': {'hvac_1': ['zone_1'], 'hvac_2': ['zone_2'], 'hvac_3': ['zone_3']}, 'zones_contain_spaces': {'zone_1': ['space_1_1', 'space_1_2'], 'zone_2': ['space_2_1'], 'zone_3': ['space_3_1']}, 'zones_contain_windows': {'zone_1': ['window_1_1', 'window_1_2'], 'zone_2': ['window_2_1', 'window_2_2'], 'zone_3': ['window_3_1', 'window_3_2', 'window_3_3